# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [75]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [76]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [77]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [78]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [79]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [80]:
# Create a Keyspace 

try:
    session.execute("""
    create keyspace if not exists udacity 
    with replication = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [81]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [82]:
# create table for query 1

query = "create table if not exists session_item "
query = query + """(sessionId int,
                    itemInSession int,
                    artist text, 
                    song text, 
                    length float, 
                    primary key (sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [83]:
# # insert data for query 1 table

query = "insert into session_item (sessionId, itemInSession, artist, song, length)"
query = query + " values (%s, %s, %s, %s, %s)"

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))
        except Exception as e:
            print(e)    

In [84]:
## verify that query 1 is correct

query = "select artist, song, length from session_item where sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [85]:
# display data in tabular format

from prettytable import PrettyTable
t = PrettyTable(['artist', 'song', 'length'])

query = "select artist, song, length from session_item where sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    t.add_row([row.artist, row.song, row.length])
print(t)

+-----------+---------------------------------+--------------------+
|   artist  |               song              |       length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [86]:
# create table for query 2

query = "create table if not exists user_session_songs"
query = query + """(userId int, 
                    sessionId int,
                    itemInSession int,
                    artist text,
                    song text,
                    firstName text,
                    lastName text,
                    primary key((userId, sessionId), itemInSession))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [87]:
# insert data for query 2 table

query = "insert into user_session_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
query = query + " values (%s, %s, %s, %s, %s, %s, %s)"

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader: 
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4])))
        except Exception as e:
            print(e)           

In [88]:
# verify that query 2 is correct

query = """select artist, song, firstName, lastName from user_session_songs
            where userId = 10 and sessionId = 182 
            order by itemInSession desc"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')


In [89]:
# display data in tabular format

from prettytable import PrettyTable
t = PrettyTable(['artist', 'song', 'firstname', 'lastname'])

query = """select artist, song, firstName, lastName from user_session_songs
            where userId = 10 and sessionId = 182 
            order by itemInSession desc"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    t.add_row([row.artist, row.song, row.firstname, row.lastname])
print(t)

+-------------------+------------------------------------------------------+-----------+----------+
|       artist      |                         song                         | firstname | lastname |
+-------------------+------------------------------------------------------+-----------+----------+
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [90]:
# create table for query 3

query = "create table if not exists song_listens"
query = query + """(song text, 
                    userId text, 
                    firstName text,
                    lastName text,
                    primary key(song, userId))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [91]:
# insert data for query 3 table

query = "insert into song_listens (song, userId, firstName, lastName)"
query = query + " values (%s, %s, %s, %s)"


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query, (line[9], line[10], line[1], line[4]))
        except Exception as e:
            print(e)

In [92]:
# verify that query 3 is correct

query = "select firstName, lastName from song_listens where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


In [93]:
# display data in tabular format

from prettytable import PrettyTable
t = PrettyTable(['firstname', 'lastname'])

query = "select firstName, lastName from song_listens where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    t.add_row([row.firstname, row.lastname])
print(t)

+------------+----------+
| firstname  | lastname |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Drop the table before closing out the sessions

In [94]:
table_names = ['session_item', 'user_session_songs', 'song_listens']

for table in table_names:
    query = "drop table " + table
    try:
        session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection

In [95]:
session.shutdown()
cluster.shutdown()